# Music name generator
Well here we are. I'm going to find THE best music name, thanks to machine learning

In [63]:
import numpy as np 
import pandas as pd 
import string
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/home/goznalo/Programming/Python/musicnames'):
    for filename in filenames:
        if ".git" not in dirname:
            print(os.path.join(dirname, filename))
            
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback

%matplotlib inline

/home/goznalo/Programming/Python/musicnames/artists.csv
/home/goznalo/Programming/Python/musicnames/clearedlist3.csv
/home/goznalo/Programming/Python/musicnames/miusic.ipynb
/home/goznalo/Programming/Python/musicnames/clearedlist1.csv
/home/goznalo/Programming/Python/musicnames/clearedlist2.csv
/home/goznalo/Programming/Python/musicnames/.ipynb_checkpoints/miusic-checkpoint.ipynb


### Importing the data
Let's import the data from the kaggle dataset music-artists-popularity, using the Pandas library.

In [34]:
filename = '/home/goznalo/Programming/Python/musicnames/artists.csv'
dataset = pd.read_csv(filename, usecols = [2], dtype=str) # Obtain the artist name column.

In [35]:
dataset.head # preview of the data

<bound method NDFrame.head of                  artist_lastfm
0                     Coldplay
1                    Radiohead
2        Red Hot Chili Peppers
3                      Rihanna
4                       Eminem
...                        ...
1466078                    NaN
1466079                    NaN
1466080                    NaN
1466081                    NaN
1466082                    NaN

[1466083 rows x 1 columns]>

### Clearing, part 1
We don't want any of the NaN entries, nor duplicate names. We will also convert it to a numpy array for posterior transformations.

In [36]:
df = dataset.dropna(axis = 0,how = 'any',thresh = None).drop_duplicates(subset=None) #removing NaN's and converting to numpy.
names = np.squeeze(np.asarray(df))

In [37]:
len(names)

957812

In [38]:
names[0]

'Coldplay'

### Clearing, part 2
We will now remove all those names using non-standard characters. What I mean by non-standard is all those characters not being latin ones, nor punctuation ones, nor digits, nor spaces. For instance, we will get rid of those using chinese characters or greek letters.

If we try applying the .isalnum() method, we quickly run into trouble as these non-latin characters return True. We need to come up with a solution. One way around this is by using the string module. With it, the string.printable characters are those we want to allow in our names, therefore we make a set out of them, which will be used against the names of the database in a loop.

In [39]:
validchars = {}
validchars[0] = set(string.printable)
print(validchars[0])

{'E', 'R', 'Q', 'z', 'J', ':', '-', 's', 'W', 'c', '0', '2', '1', 'p', 'b', '6', '}', '3', '9', 'r', '\t', '^', 'g', ';', '"', 'F', '.', '/', '!', '\x0c', 'P', 'x', 'h', 'S', 'e', 'Z', 'L', '\r', 'O', '@', '\\', '8', 'j', 'n', 'w', "'", 'm', 'N', '?', '*', '%', 'q', '\x0b', '5', '#', 'H', 'v', 'l', 'i', 'I', '`', 'A', 'U', 'k', 'X', '$', 't', '4', '\n', 'o', 'C', '=', 'u', '&', ',', ' ', '7', 'V', '>', '<', 'y', ']', 'f', '[', 'G', 'M', '_', '{', 'T', '~', ')', 'Y', 'K', 'D', 'a', 'd', '|', 'B', '+', '('}


Now comes the loop. An initial approach was creating a new object, a numpy array, to which we would append each valid name, therefore discarding the rest of the names with invalid characters. However, the appending operation makes the algorithm take exponential time of completion. Instead, we can save the index of each invalid word in a list, which we then feed to the np.delete() function to remove those entries of the "names" array

In [40]:
clearedlist = {}
def deletechars(validcharacters, listofnames, verbose=False):  # We define it within a function, as we will need it in the next steps
    i = 0 # counts each iteration
    j = 0 # counts each invalid word
    deletelist= []
    for name in listofnames:
        if not all(char in validcharacters for char in name):
            deletelist.append(i)
            if j%10000 == 0 and verbose:
                print("invalid: " + name)
            j += 1
        i += 1
        if i%50000 == 0 and verbose:
            print(str(i) + " cases inspected.")
    return np.delete(listofnames, deletelist)

clearedlist[0] = deletechars(validchars[0], names) #by default, verbose = False (no output)

In [41]:
clearedlist[0][0:22] # Check that Beyoncé has been correctly removed.

array(['Coldplay', 'Radiohead', 'Red Hot Chili Peppers', 'Rihanna',
       'Eminem', 'The Killers', 'Kanye West', 'Nirvana', 'Muse', 'Queen',
       'Foo Fighters', 'Linkin Park', 'Lady Gaga', 'The Rolling Stones',
       'Daft Punk', 'Green Day', 'Katy Perry', 'The Beatles', 'Oasis',
       'Gorillaz', 'Michael Jackson', 'Maroon 5'], dtype=object)

In [42]:
np.savetxt("clearedlist1.csv", clearedlist[0], delimiter=",", fmt='%s') #save the list to a csv file.

### Clearing, part 3: choose your own adventure
We will now distinguish four different cases which can be studied, with decreasing complexity. 
1. The full list, as is.
2. The list, having removed punctuation characters and digits.
3. The list, having removed both punctuation characters, digits and names with more than two words.
4. The list, having removed both punctuation characters, digits and names with more than one word.

Case 2 is easy to implement, just applying the previously defined deletechars() function, specifying punctuation characters as invalid (allowing whitespaces). Case 4 is also straightforward to implement, as string.ascii_letters considers any whitespace invalid. Case 3 entails splitting each name using a "space" delimiter, and then discarding them lengthwise.

In [43]:
case = 3 #change to whichever of the above
verbose = False

In [44]:
validchars[1] = set(string.ascii_letters).union(" ")
validchars[2] = set(string.ascii_letters).union(" ") #we won't use it here, but we will count the characters with this.
validchars[3] = set(string.ascii_letters)

clearedlist[1] = []
clearedlist[2] = []
clearedlist[3] = []

if (case == 2 or case == 3 or case == 4):
    clearedlist[1] = deletechars(validchars[1], clearedlist[0])
    np.savetxt("clearedlist2.csv", clearedlist[1], delimiter=",", fmt='%s')
    
    if (case == 3 or case == 4): #as the function above, but indexing based on the length of string.split()
        i = 0 
        j = 0 
        deletelist = []
        for name in clearedlist[1]:
            if len(name.split()) > 2:
                deletelist.append(i)
                if j%10000 == 0 and verbose:
                    print("invalid: " + name)
                j += 1
            i += 1
            if i%50000 == 0 and verbose:
                print(str(i) + " cases inspected (lengthwise).")
        clearedlist[2] = np.delete(clearedlist[1], deletelist)
        np.savetxt("clearedlist3.csv", clearedlist[2], delimiter=",", fmt='%s') 
        
        if case == 4: # string.ascii_letters considers whitespaces as invalid.
            clearedlist[3] = deletechars(validchars[3], clearedlist[2])
            np.savetxt("clearedlist4.csv", clearedlist[3], delimiter=",", fmt='%s')

print((len(names), len(clearedlist[0]), len(clearedlist[1]), len(clearedlist[2]), len(clearedlist[3])))
print(clearedlist[2][0:22])

(957812, 861756, 743526, 626581, 0)
['Coldplay' 'Radiohead' 'Rihanna' 'Eminem' 'The Killers' 'Kanye West'
 'Nirvana' 'Muse' 'Queen' 'Foo Fighters' 'Linkin Park' 'Lady Gaga'
 'Daft Punk' 'Green Day' 'Katy Perry' 'The Beatles' 'Oasis' 'Gorillaz'
 'Michael Jackson' 'Arctic Monkeys' 'Drake' 'David Bowie']


### Some more pre-processing, I guess

Now that we have our desired array of names, we need to prepare them for being inputted to a neural network. There are several steps to be taken: first, 
we will append to each name a "\n" character: this will be our end-of-name character. We will later make all names lowercase, so as to use a dimensionally smaller encoding. Then, making use of the char_to_index dictionary and the keras.utils.to_categorical() function, we will create our one-hot vectors


In [45]:
data = clearedlist[case-1][0:int(round(len(clearedlist[case-1])/10))] #definitive list of names
m = len(data) #training examples

for i in range(m):
    data[i] = data[i].lower()

In [46]:
for i in range(len(data)):
    data[i] = data[i] + '\n'

In [56]:
data[1]

'radiohead\n'

In [57]:
max_char = len(max(data, key=len))
print(max_char)

44


In [58]:
characters = validchars[case-1].union('\n')
chars = sorted(list(dict.fromkeys([char.lower() for char in characters])))
numchars = len(chars)

print(numchars)

28


In [59]:
char_to_index = { ch:ix for ix, ch in enumerate(chars)} #assigns numbers to characters.
index_to_char = { ix:ch for ix, ch in enumerate(chars)} #assigns numbers to characters.

print(char_to_index)
print(index_to_char)

{'\n': 0, ' ': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}
{0: '\n', 1: ' ', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e', 7: 'f', 8: 'g', 9: 'h', 10: 'i', 11: 'j', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'q', 19: 'r', 20: 's', 21: 't', 22: 'u', 23: 'v', 24: 'w', 25: 'x', 26: 'y', 27: 'z'}


In [60]:
print((m,max_char,numchars))

(62658, 44, 28)


In [61]:
X = np.zeros((m, max_char, numchars))
Y = np.zeros((m, max_char, numchars))

for i in range(m):
    name = list(data[i])
    for j in range(len(name)):
        X[i, j, char_to_index[name[j]]] = 1
        if j < len(name)-1:
            Y[i, j, char_to_index[name[j+1]]] = 1

In [64]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_char, numchars), return_sequences=True))
model.add(Dense(numchars, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [65]:
def make_name(model):
    name = []
    x = np.zeros((1, max_char, numchars))
    end = False
    i = 0
    
    while end==False:
        probs = list(model.predict(x)[0,i])
        probs = probs / np.sum(probs)
        index = np.random.choice(range(numchars), p=probs)
        if i == max_char-2:
            character = '\n'
            end = True
        else:
            character = index_to_char[index]
        name.append(character)
        x[0, i+1, index] = 1
        i += 1
        if character == '\n':
            end = True
    
    print(''.join(name))

In [66]:
def generate_name_loop(epoch, _):
    if epoch % 25 == 0:
        
        print('Names generated after epoch %d:' % epoch)

        for i in range(3):
            make_name(model)
        
        print()

In [67]:
filepath="checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
name_generator = LambdaCallback(on_epoch_end = generate_name_loop)
callbacks_list = [checkpoint, name_generator]

In [69]:
model.fit(X, Y, batch_size=512, epochs=51, callbacks = callbacks_list, verbose=1)

Epoch 1/51
123/123 [==============================] - ETA: 0s - loss: 0.6486Names generated after epoch 0:
kq ufannar

v

wimeo


123/123 [==============================] - 29s 238ms/step - loss: 0.6486
Epoch 2/51
 47/123 [==========>...................] - ETA: 17s - loss: 0.5999

KeyboardInterrupt: 